RoBERTa Model Implementation for Classification (using sliding window approach)

In [1]:
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/spyman/Documents/Deep Learning/LAW-AI classification/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def sliding_window_tokenize(texts, tokenizer, max_length=512, stride=256):
    encodings = []
    for text in texts:
        tokens = tokenizer.tokenize(text)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        for start in range(0, len(token_ids), stride):
            end = min(start + max_length, len(token_ids))
            input_ids = token_ids[start:end]
            attention_mask = [1] * len(input_ids)
            padding_length = max_length - len(input_ids)
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            encodings.append({'input_ids': input_ids, 'attention_mask': attention_mask})
    return encodings

class SlidingWindowDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx // len(self.encodings[0])])
        return item

    def __len__(self):
        return len(self.labels) * len(self.encodings[0])

In [3]:
import pandas as pd

df = pd.read_csv('Input.csv') 
texts = df['Text'].tolist()
labels = df['Label'].tolist()

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, encoded_labels, test_size=0.1)

train_encodings = sliding_window_tokenize(train_texts, tokenizer, max_length=512, stride=256)
val_encodings = sliding_window_tokenize(val_texts, tokenizer, max_length=512, stride=256)

train_dataset = SlidingWindowDataset(train_encodings, train_labels)
val_dataset = SlidingWindowDataset(val_encodings, val_labels)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

  3%|▎         | 10/342 [02:13<1:12:19, 13.07s/it]

{'loss': 1.3982, 'grad_norm': 2.9817054271698, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


  6%|▌         | 20/342 [04:17<1:06:54, 12.47s/it]

{'loss': 1.3738, 'grad_norm': 3.302122116088867, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.18}


  9%|▉         | 30/342 [06:22<1:05:18, 12.56s/it]

...

 99%|█████████▉| 340/342 [1:31:31<00:23, 11.67s/it]

{'loss': 1.3796, 'grad_norm': 2.6450376510620117, 'learning_rate': 3.4000000000000007e-05, 'epoch': 2.98}


100%|██████████| 342/342 [1:31:55<00:00, 16.13s/it]

{'train_runtime': 5515.9664, 'train_samples_per_second': 0.496, 'train_steps_per_second': 0.062, 'train_loss': 1.388234747780694, 'epoch': 3.0}


TrainOutput(global_step=342, training_loss=1.388234747780694, metrics={'train_runtime': 5515.9664, 'train_samples_per_second': 0.496, 'train_steps_per_second': 0.062, 'total_flos': 719884774342656.0, 'train_loss': 1.388234747780694, 'epoch': 3.0})

In [8]:
from collections import Counter
import torch

model.save_pretrained('./fine-tuned-roberta512-sliding-window')
tokenizer.save_pretrained('./fine-tuned-roberta512-sliding-window')

def predict(texts):
    predictions = []
    for text in texts:
        encodings = sliding_window_tokenize([text], tokenizer, max_length=512, stride=256)
        text_predictions = []
        for enc in encodings:
            inputs = {key: torch.tensor(val).unsqueeze(0) for key, val in enc.items()}
            with torch.no_grad():
                outputs = model(**inputs)
            pred = torch.argmax(outputs.logits, dim=1).tolist()
            text_predictions.append(pred[0])
        most_common_pred = Counter(text_predictions).most_common(1)[0][0]
        predictions.append(most_common_pred)
    return predictions

example_texts = ['''A property owner files a lawsuit against their neighbor for encroaching on their land and demands that the boundary be restored to its rightful place.''']
predictions = predict(example_texts)
decoded_predictions = [label_encoder.inverse_transform([p])[0] for p in predictions]
print(decoded_predictions)

['Civil Case']


In [9]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from collections import Counter
import joblib
import torch
tokenizer = RobertaTokenizer.from_pretrained('./fine-tuned-roberta512-sliding-window')
model = RobertaForSequenceClassification.from_pretrained('./fine-tuned-roberta512-sliding-window')
label_encoder = joblib.load('label_encoder.joblib')

In [5]:
def sliding_window_tokenize(texts, tokenizer, max_length=512, stride=256):
    encodings = []
    for text in texts:
        tokens = tokenizer.tokenize(text)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        for start in range(0, len(token_ids), stride):
            end = min(start + max_length, len(token_ids))
            input_ids = token_ids[start:end]
            attention_mask = [1] * len(input_ids)
            padding_length = max_length - len(input_ids)
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            encodings.append({'input_ids': input_ids, 'attention_mask': attention_mask})
    return encodings

In [10]:
def predict(texts):
    predictions = []
    for text in texts:
        encodings = sliding_window_tokenize([text], tokenizer, max_length=512, stride=256)
        text_predictions = []
        for enc in encodings:
            inputs = {key: torch.tensor(val).unsqueeze(0) for key, val in enc.items()}
            with torch.no_grad():
                outputs = model(**inputs)
            pred = torch.argmax(outputs.logits, dim=1).tolist()
            text_predictions.append(pred[0])
        most_common_pred = Counter(text_predictions).most_common(1)[0][0]
        predictions.append(most_common_pred)
    return predictions

example_texts = ['''A property owner files a lawsuit against their neighbor for encroaching on their land and demands that the boundary be restored to its rightful place.''']
predictions = predict(example_texts)
decoded_predictions = [label_encoder.inverse_transform([p])[0] for p in predictions]
print(decoded_predictions)

['Civil Case']
